In [2]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




In [3]:
# load data
data_dict = {
    "fleet_path":'data/fleet_small.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}


ports_df = pd.read_csv(data_dict['ports_path'])
fleet_df = pd.read_csv(data_dict['fleet_path'])
distance_matrix = pd.read_csv(data_dict['dm_path'])




In [4]:
from env.env import CarbonEnv
from utils.utils import func_ballast

envo = CarbonEnv()
con_df, con_tensor = envo.create_contracts_tensor()

2021-11-05 19:14:27.628596: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 19:14:27.629794: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
envo.state

{'contracts_state': <tf.Tensor: shape=(1, 4, 10), dtype=float32, numpy=
 array([[[3.0000e+00, 8.0000e+00, 2.0000e+00, 0.0000e+00, 4.0000e+00,
          5.4474e+04, 5.0000e+00, 1.0000e+00, 2.8700e+02, 3.0000e+00],
         [2.0000e+00, 8.0000e+00, 4.0000e+00, 0.0000e+00, 9.0000e+00,
          7.4757e+04, 1.0000e+01, 1.0000e+00, 1.4600e+03, 1.1000e+01],
         [1.0000e+01, 5.0000e+00, 2.0000e+00, 0.0000e+00, 5.0000e+00,
          5.0991e+04, 6.0000e+00, 1.0000e+00, 4.2500e+02, 4.0000e+00],
         [1.0000e+00, 1.0000e+01, 4.0000e+00, 0.0000e+00, 5.0000e+00,
          7.5529e+04, 6.0000e+00, 1.0000e+00, 6.0000e+02, 8.0000e+00]]],
       dtype=float32)>,
 'ships_state': <tf.Tensor: shape=(1, 4, 11), dtype=float32, numpy=
 array([[[1.0000000e+00, 5.0000000e+04, 5.3851833e+00, 0.0000000e+00,
          3.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
          1.6530000e+03, 1.2400000e+02, 5.8500000e+02],
         [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
    

In [5]:
from utils.utils import cii_expected

In [6]:
fl['cii_threshold'] = fl['dwt'].map(cii_expected)

In [7]:
fl['current_port'] = np.random.randint(1,11,4)

In [3]:
for i in range(0,8):
    
    day = i // 4
    print(day)

0
0
0
0
1
1
1
1


In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type

In [ ]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

# cargo_size vs ship_shize

contracts = create_contrs(seed=56)

# gia ka8e contract se spawned contracts

contract = contracts.loc[0]

fleet_df
# elegxw ka8e ploio pou einai available dhladh exei availability 1 
mask_of_available_ships = fleet_df['ship_availability'] == 1

available_ships = fleet_df[mask_of_available_ships]

available_ships_numbers = available_ships.ship_number

available_ships_numbers



# checkare sto masking an kaneis mask out tous assous h ta mhdenika

# gia ka8e tetoio ploio


# get ship_number

ship_numbers = 
agent_ship_selection = 2

# get ship info
ship_mask = fleet_df['ship_number'] == agent_ship_selection

ship_info = fleet_df[ship_mask]
print(ship_info)

ship_current_port = ship_info.current_port
print(ship_current_port)






start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_distance = find_distance(ship_current_port,start_port_number)
print(trip_1_distance)
trip_2_distance = find_distance(start_port_number,end_port_number)
print(trip_2_distance)
trip_total_distance = trip_1_distance + trip_2_distance




trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")
